In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


# モデル読み込み

In [ ]:
from model.Temperature.Laser import LaserProfile

# テスト

In [ ]:
# 初期化
laser_const = LaserProfile()

In [ ]:
dir(laser_const)

In [ ]:
# linearのwidthが、上のwidthより小さい場合
# 最後が定常加熱になる
# base_widthとlinear_widthを実装する。
# base >= linearである必要がある
# base < linearでもレーザー出てたような。エネルギーの上がり方が途中までになる？

# 定常加熱の場合

## パラメータ設定

In [ ]:
# 設定するパラメータ
# 辞書にする
linear = False
power_W = 10
base_width_ms = 8_000
whole_time_ms = 10_000
laser_diameter_um = 17.5

In [ ]:
# クラスにつける
laser_const.linear = linear
laser_const.power_W = power_W
laser_const.base_width_ms = base_width_ms
laser_const.whole_time_ms = whole_time_ms
laser_const.laser_diameter_um = laser_diameter_um

In [ ]:
laser_const.get_laser_profile_arr()

In [ ]:
plt.plot(laser_const.time_arr, laser_const.power_arr)
plt.scatter(laser_const.time_arr, laser_const.power_arr)


# 線形加熱の場合

In [ ]:
# 初期化
laser_linear = LaserProfile()

## パラメータ設定

In [ ]:
# 設定するパラメータ
params_dict = {
    'linear': True,
    'base_width_ms': 18_000,
    'linear_width_ms': 18_000,
    'whole_time_ms': 20_000,
    'laser_diameter_um': 17.5,
    'start_power_W': 14,
    'goal_power_W': 50,
    'step_width_ms': 100,
}

In [ ]:
params_list = list(params_dict.keys())

In [ ]:
for param in params_list:
    print(f"{param}: {params_dict[param]}")

In [ ]:
for param in params_list:
    exec(f"laser_linear.{param} = {params_dict[param]}")
    exec(f"print(laser_linear.{param})")

In [ ]:
laser_linear.get_laser_profile_arr()

In [ ]:
plt.plot(laser_linear.time_arr, laser_linear.power_arr)
plt.scatter(laser_linear.time_arr, laser_linear.power_arr, s=1)

## 最後だけ定常する場合

In [ ]:

# 設定するパラメータ
params_dict = {
    'linear': True,
    'base_width_ms': 18_000,
    'linear_width_ms': 15_000,
    'whole_time_ms': 20_000,
    'laser_diameter_um': 17.5,
    'start_power_W': 0,
    'goal_power_W': 15,
    'step_width_ms': 100,
}

params_list = list(params_dict.keys())
for param in params_list:
    print(f"{param}: {params_dict[param]}")
for param in params_list:
    exec(f"laser_linear.{param} = {params_dict[param]}")
    exec(f"print(laser_linear.{param})")
laser_linear.get_laser_profile_arr()


In [ ]:

plt.plot(laser_linear.time_arr, laser_linear.power_arr)
plt.scatter(laser_linear.time_arr, laser_linear.power_arr, s=1)


# develop

In [ ]:
laser.step_num

In [ ]:
laser.energy_step

In [ ]:
laser.time_series/1000

In [ ]:
laser.time_series.shape

In [ ]:
x = np.arange(laser.time_series.shape[0])
plt.scatter(x, laser.time_series/1000)
plt.xlim(0, 10)

In [ ]:
# 時間配列

# start
time_arr = np.zeros(1)

# 加熱はじめ
time_arr = np.r_[time_arr, laser.delay + laser.time_adjust]
time_arr = np.r_[time_arr, laser.delay + laser.time_adjust]

# step加熱
# FIXME
#   time_adjustは一回にする
#   time_series[i+1]がわかりづらい。計算する
for i in range(int(laser.step_num)):
    time_arr = np.r_[time_arr, laser.time_series[i+1]+ laser.time_adjust]
    time_arr = np.r_[time_arr, laser.time_series[i+1]+ laser.time_adjust]

# 加熱終わり
time_arr = np.r_[time_arr, time_arr[-1]] # クエンチ
end_time = laser.delay + laser.width + laser.residue
time_arr = np.r_[time_arr, end_time]

time_arr /= 1000
time_arr

In [ ]:
# エネルギー配列

# start
energy_arr = np.zeros(1)

# 加熱はじめ
energy_arr = np.r_[energy_arr, np.array([0])]
energy_arr = np.r_[energy_arr, laser.start_power]

# step加熱
energy = laser.start_power
for i in range(int(laser.step_num)):
    energy_arr = np.r_[energy_arr, energy]
    energy += laser.energy_step
    energy_arr = np.r_[energy_arr, energy]

# 加熱終わり
energy_arr = np.r_[energy_arr, np.zeros(1)]
energy_arr = np.r_[energy_arr, np.zeros(1)]

energy_arr

In [ ]:
plt.figure(figsize=(6,6), dpi=1000)
plt.plot(time_arr, energy_arr, lw=0.5)  

# クラスの方の配列をplotして確認

In [ ]:
plt.figure(figsize=(6,6), dpi=1000)
plt.plot(laser.time_arr, laser.energy_arr, lw=0.5)  
